# TF Eager Tutorial (Define by Run)

정리 및 요약 by Ryah Shin

[참고1: 구글 블로그](https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html)

[참고2: 구글 깃허브](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/g3doc/guide.md)

[Eager 모드 설치방법(TF Nightly)](https://github.com/tensorflow/tensorflow#installation)

##  Eager Execution

 Eager (이거)는 기존의 세션과 그래프 형식의 작동 방식에서 명령형 (imperative) 스타일로 텐서플로우를 활용할수 있는 기능이다.  기존에 텐서플로우 방식이 세션과 그래프 완성 후 디버깅을 해야하는 불편함이 있었다면, 이거 모드를 통해서 좀 더 직관적으로 접근 가능하다.
 
텐서플로우 2018 Dev Summit에서 내용을 발췌하면 다음과 같다.

 - Eager 모드는 점점 contrib에서 정식으로 변하고 있고 session을 사용하지 않고 코드를 실행 가능하다.
 - 그래디언트 계산을 손쉽게 수정이 가능하다.
 - Dataset (텐서플로우 자료 구조 중 하나)를 통해서 sqlite database를 읽을 수 있다 (아직 실험적)
 - TensorRT를 지원함으로써 모델을 최적화 할 수 있다.
 
이 장에서는 Eager 모드에 대한 간략한 소개, 기능 설명 및 예제를 제공하겠다.

In [2]:
import tensorflow as tf

# eager 모드를 실행하는 방법
# eager 모드는 프로그램 시작 때 실행을 해야하며, 재시작 까지는 eager 모드가 유지됩니다.
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.8.0
Eager execution: True


grad() function return the derivative of loss with respect to weight and bias. Then passing this to optimizer.apply_gradients() completes the process of apply gradient descent.

Apart from the above changes, almost everything remains same. Here are some important points before using eager execution:

· Data must be initialized using tensorflow.data.Dataset. (Use can use other ways but they are not recommended)

· Eager execution runs by default on CPU, to use GPU include below code:
with tf.device(‘/gpu:0’)

· Eager execution doesn’t create Tensor Graph, to build graph just remove the tf.enable_eager_execution().

· Eager execution is good for R&D but for production you should use graph execution.

· You can save the model generated with eager execution and later load this model in graph or eager execution.

Here is the link to the working code on Google Colaboratory which you can try and play with.

Update: Definition of grad() function.

In [3]:
import numpy as np

# 직관적인 연산이 가능합니다.

 - Eager 모드를 통해, 조금 더 직관적으로 연산이 가능하다. 2 * 2 매트릭스에서 연산을 하는 예제들을 확인하겠습니다.


In [4]:
x = tf.matmul([[1, 2],
               [3, 4]],
              [[4, 5],
               [6, 7]])

y = tf.add(x, 1)

print(x)
print(y)

tf.Tensor(
[[16 19]
 [36 43]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[17 20]
 [37 44]], shape=(2, 2), dtype=int32)


In [5]:
x.numpy() #넘파이 형태로 변환도 가능합니다.

array([[16, 19],
       [36, 43]], dtype=int32)

In [6]:
#반대로 tf.constant 활용하여 넘파이에서 텐서플로우 구조로도 변화가 가능합니다.

np_val = np.array(10., dtype=np.float32)
tf_val = tf.constant(np_val) #tf.constant를 사용하여 넘파이 데이터 변환

print(np_val)
print(tf_val)

10.0
tf.Tensor(10.0, shape=(), dtype=float32)


## Define and Print Tensorflow Variables

In [31]:
x = tf.get_variable(name="x", shape=[], dtype=tf.float32, initializer=tf.zeros_initializer)
print(x)

#Tensorflow의 변수는 tensor로 나타냄으로, read_value()를 통해 현재 값으로 접근이 가능함.
#Tensorflow의 함수는 자동으로 초기화

<tf.Variable 'x:0' shape=() dtype=float32, numpy=0.0>


In [33]:
#numpy를 통한 변환
print(x.read_value().numpy())

#Tensorflow변수의 값을 변경하기
x.assign(42)
print(x.assign)

x.assign_add(3) #x 값에 더하기 적용
print(x.read_value())

print(x + 3) #텐서 변수를 자유자제로 활용해보기

print(x * [1, 2, 4]) #자동으로 broadcasting도 가능함

45.0
<bound method ResourceVariable.assign of <tf.Variable 'x:0' shape=() dtype=float32, numpy=42.0>>
tf.Tensor(45.0, shape=(), dtype=float32)
tf.Tensor(48.0, shape=(), dtype=float32)
tf.Tensor([ 45.  90. 180.], shape=(3,), dtype=float32)


## Automatic Difference (Gradients)

 - tfe.gradients_function(f): 입력 f에 대해 arg 미분값을 돌려준다.
 - tfe.value_and_gradients_function(f): tfe.gradients_function(f)과 비슷하지만, 함수가 들어오면 이전 f값과 미분값에 대해 값을 출력한다.

In [34]:
def square(x):
    return tf.multiply(x, x)
assert 9 == square(3.).numpy()

grad = tfe.gradients_function(square)
assert 6 == grad(3.)[0].numpy()

print(square(3.))
print(grad(3.)) #x^2 -> 2x -> 6

tf.Tensor(9.0, shape=(), dtype=float32)
[<tf.Tensor: id=14844, shape=(), dtype=float32, numpy=6.0>]


In [35]:
#2차 gradients_function
grad2 = tfe.value_and_gradients_function(lambda x: grad(x)[0])
#assert 2 == grad2(3.)[0].numpy()
print("2nd grad: {}".format(grad2(3.)))

#3차 grad.
grad3 = tfe.gradients_function(lambda x: grad2(x)[0])
#assert 0 == grad3(3.)[0].numpy()
print(grad3(3.))

#absolute value
def abs(x):
    return x if x > 0. else -x

grad = tfe.gradients_function(abs)

print(grad(2.0))  # [1.]
print(grad(-2.0)) # [-1.]

2nd grad: (<tf.Tensor: id=14851, shape=(), dtype=float32, numpy=6.0>, [<tf.Tensor: id=14856, shape=(), dtype=float32, numpy=2.0>])
[<tf.Tensor: id=14875, shape=(), dtype=float32, numpy=2.0>]
[<tf.Tensor: id=72, shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: id=14887, shape=(), dtype=float32, numpy=-1.0>]


In [36]:
#실제 linear regression을 통하여 활용해보자

def prediction(input, weight, bias):
    return input * weight + bias

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# A loss function: Mean-squared error
def loss(weight, bias):
    error = prediction(training_inputs, weight, bias) - training_outputs
    return tf.reduce_mean(tf.square(error))

# Function that returns the the derivative of loss with respect to
# weight and bias
grad = tfe.gradients_function(loss)

# Train for 200 steps (starting from some random choice for W and B, on the same
# batch of data).
W = 5.
B = 10.
learning_rate = 0.01
print("Initial loss: %f" % loss(W, B).numpy())
for i in range(200):
    (dW, dB) = grad(W, B)
    W -= dW * learning_rate
    B -= dB * learning_rate
    if i % 20 == 0:
        print("Loss at step %d: %f" % (i, loss(W, B).numpy()))
print("Final loss: %f" % loss(W, B).numpy())
print("W, B = %f, %f" % (W.numpy(), B.numpy()))

Initial loss: 69.151985
Loss at step 0: 66.454216
Loss at step 20: 30.185934
Loss at step 40: 14.019278
Loss at step 60: 6.812951
Loss at step 80: 3.600699
Loss at step 100: 2.168821
Loss at step 120: 1.530553
Loss at step 140: 1.246040
Loss at step 160: 1.119215
Loss at step 180: 1.062682
Final loss: 1.038317
W, B = 3.034794, 2.132014


# Why

## Building and training models

 - eager에서는 특별히 수정해야 하지 않는 한, tf.layers와 같은 모듈을 사용을 권장함
 - Optimizer와 layer를 간단하게 정리

## Variable & Optimization

 - tfe.Variable: 변형가능한 Tensor값을 저장하는 객체로써, 학습이나 미분을 할때 값에 대한 access가 가능함. 모델의 파라메터들이 python변수에 저장 될 수 있다는 이야기임
 - tfe.gradients_function(f): 쉬운 미분을 지원하지만, 모든 파라메터들이 f와 연동이 되어있어야 하여, 학습시 큰 파라메터에 대한 대응이 힘듬
 - tfe.implicit_gradients: 비슷한 기능이지만 몇가지 특수 기능이 있음?

In [17]:
#실제 linear regression을 통하여 활용해보자

class Model(object):
    def __init__(self):
        self.W = tfe.Variable(5., name='weight')
        self.B = tfe.Variable(10., name='bias')

    def predict(self, inputs):
        return inputs * self.W + self.B


# The loss function to be optimized
def loss(model, inputs, targets):
    error = model.predict(inputs) - targets
    return tf.reduce_mean(tf.square(error))

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# Define:
# 1. A model
# 2. Derivatives of a loss function with respect to model parameters
# 3. A strategy for updating the variables based on the derivatives
model = Model()
grad = tfe.implicit_gradients(loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

# The training loop
print("Initial loss: %f" %
      loss(model, training_inputs, training_outputs).numpy())
for i in range(201):
    optimizer.apply_gradients(grad(model, training_inputs, training_outputs))
    if i % 20 == 0:
        print("Loss at step %d: %f" %
              (i, loss(model, training_inputs, training_outputs).numpy()))
print("Final loss: %f" % loss(model, training_inputs, training_outputs).numpy())
print("W, B = %s, %s" % (model.W.numpy(), model.B.numpy()))

Initial loss: 67.940033
Loss at step 0: 65.334366
Loss at step 20: 30.092083
Loss at step 40: 14.164181
Loss at step 60: 6.957934
Loss at step 80: 3.694184
Loss at step 100: 2.214460
Loss at step 120: 1.542879
Loss at step 140: 1.237761
Loss at step 160: 1.098992
Loss at step 180: 1.035816
Loss at step 200: 1.007025
Final loss: 1.007025
W, B = 3.0174496, 2.1403313


# Building Models (개선필요)

MNIST 2 Layer모델을 간단하게 Class로 만드는 예제

tfe.Network: 기본적으로 layer의 Container역할을 하여, 다른 NW객체에 임비디드 되어 NW객체가 된다.

추가로, inspection, saving, & restoring에 도움을 준다.

In [18]:
class MNISTModel(tfe.Network):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = self.track_layer(tf.layers.Dense(units=10))
        self.layer2 = self.track_layer(tf.layers.Dense(units=10))
    def call(self, input):
        """모델 실행"""
        result = self.layer1(input)
        result = self.layer2(result)
        return result
    
#placeholder나 session에 대한 기능이 없고, input을 pass되면 자동으로 세팅 됨

In [19]:
# 테스트 데이터셋 생성하기
model = MNISTModel()
batch = tf.zeros([1, 1, 784])
print(batch.shape)
result = model(batch)
print(result)

(1, 1, 784)
tf.Tensor([[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]], shape=(1, 1, 10), dtype=float32)


In [20]:
#학습을 위한 loss func, grad, 그리고 업데이트

#1. loss func
def loss_function(model, x, y):
    y_ = model(x)
    return tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_)

#2. training loop
#implicit_gradients(): 모든 TF 값에 대한 미분을 계산한다.

optimizer = tf.train.GradientDescentOptimizer(learning_rate =0.001)
for (x, y) in tfe.Iterator(dataset):
    grads = tfe.implicit_gradients(loss_function)(model, x, y)
    optimizer.apply_gradients(grads)

#GPU사용하기
#optimizer.min을 통해서, 짧게 작성하였지만, apply_gradients()기능을 써도 가능

with tf.device("/gpu:0"):
    for (x, y) in tfe.Iterator(dataset):
        optimizer.minimize(lambda: loss_function(model, x, y))

NameError: name 'dataset' is not defined

## Debug Errors with Instant Feedback

- runtime 이슈들을 디버깅하고, 코드에 관련된 사항을 interactive하게 볼 수 있음
- 간단하게 4 vector와 2개의 tf.slice()를 활용하여, 정상 케이스와 에러 케이스를 분류 하겠음

In [21]:
vector = tf.constant([10.0, 20.0, 30.0, 40.0])

In [22]:
#정상 케이스: 인덱스 값이 내부에 적용이 되기 때문
print(tf.slice(vector, [1], [3]))

tf.Tensor([20. 30. 40.], shape=(3,), dtype=float32)


In [23]:
#비정상 케이스: 인덱스 값이 데이터에서 넘가기 때문에
try:
    print(tf.slice(vector, [1], [4]))
except tf.OpError as e:
    print("Caught error: %s" % e)

Caught error: Expected size[0] in [0, 3], but got 4 [Op:Slice]


## GPU 사용하기

 - eager에서는 GPU가 자동으로 실행되지 않기 때문에, 지정해주고 사용해야 한다.
 - .gpu()를 활용함

In [24]:
# The example code from here on will work only if your notebook
# is running on a machine with a functional CUDA GPU. The following
# line checks that.
is_gpu_available = tfe.num_gpus() > 0

# Create some Tensors
SIZE = 1000
cpu_tensor = tf.random_normal([SIZE, SIZE])

if is_gpu_available:
    gpu_tensor = cpu_tensor.gpu()
else:
    print("GPU not available.")

GPU not available.


In [25]:
# Time a CPU-based matrix multiplication

print("Time to conduct matmul on CPU:")
%time tf.matmul(cpu_tensor, cpu_tensor)

Time to conduct matmul on CPU:
CPU times: user 68.7 ms, sys: 5.09 ms, total: 73.7 ms
Wall time: 33.2 ms


<tf.Tensor: id=14745, shape=(1000, 1000), dtype=float32, numpy=
array([[ 35.7918    ,  -4.655777  , -48.272617  , ...,  -7.863409  ,
         -1.6915245 , -19.592188  ],
       [-40.651207  ,  22.911709  ,  65.95149   , ..., -63.59874   ,
        -11.126056  ,   7.5989428 ],
       [ 20.382114  , -34.620655  ,  16.751368  , ...,  15.118157  ,
        -29.63465   ,  24.727722  ],
       ...,
       [ -0.70765567,  -2.8995218 ,  16.955648  , ..., -30.425877  ,
         -5.4200916 ,  39.877968  ],
       [ 52.01768   ,  27.110682  ,  38.184597  , ...,  28.810797  ,
        -65.800735  ,   7.7899466 ],
       [-24.28569   , -55.74654   ,   4.722617  , ..., -11.698205  ,
        -23.088879  , -31.178162  ]], dtype=float32)>

In [26]:
# Time GPU-based matrix multiplications.

if is_gpu_available:
    # First use of the GPU will be slow:
    print("Time to conduct first matmul on GPU:")
    %time tf.matmul(gpu_tensor, gpu_tensor)
    print()

    # Subsequent uses are much faster:
    print("Time to conduct second matmul on GPU:")
    %time tf.matmul(gpu_tensor, gpu_tensor)

In [27]:
# Second timing demo for GPUs, after it has been used once:

cpu_tensor = tf.random_normal([SIZE, SIZE])
print("Time to conduct CPU matmul:")
%time tf.matmul(cpu_tensor, cpu_tensor)
print()

if is_gpu_available:
    gpu_tensor = cpu_tensor.gpu()
    print("Time to conduct GPU matmul:")
    %time tf.matmul(gpu_tensor, gpu_tensor)

Time to conduct CPU matmul:
CPU times: user 69.5 ms, sys: 1.77 ms, total: 71.3 ms
Wall time: 26 ms



In [28]:
#1000*1000 매트릭스 계산하기

import time

def measure(x):
    # The very first time a GPU is used by TensorFlow, it is initialized.
    # So exclude the first run from timing.
    tf.matmul(x, x)

    start = time.time()
    for i in range(10):
        tf.matmul(x, x)
    end = time.time()

    return "Took %s seconds to multiply a %s matrix by itself 10 times" % (end - start, x.shape)

# Run on CPU:
with tf.device("/cpu:0"):
    print("CPU: %s" % measure(tf.random_normal([1000, 1000])))

# If a GPU is available, run on GPU:
if tfe.num_gpus() > 0:
    with tf.device("/gpu:0"):
        print("GPU: %s" % measure(tf.random_normal([1000, 1000])))

CPU: Took 0.25575995445251465 seconds to multiply a (1000, 1000) matrix by itself 10 times


In [29]:
#Tensor 객체를 활용하여, 각 디바이스 별로 활용하는 것이 가능하다

x = tf.random_normal([10, 10])

x_gpu0 = x.gpu()
x_cpu = x.cpu()

_ = tf.matmul(x_cpu, x_cpu)  # Runs on CPU
_ = tf.matmul(x_gpu0, x_gpu0)  # Runs on GPU:0

if tfe.num_gpus() > 1:
    x_gpu1 = x.gpu(1)
    _ = tf.matmul(x_gpu1, x_gpu1)  # Runs on GPU:1

RuntimeError: Error copying tensor to device: GPU:0. GPU:0 unknown device.

## Using Eagar with Graphs

- Eagar 자체는 개발하고 디버깅 할 때 좋은 기능을 가지지만, Tensorflow graph형식이 분산 학습, 성능 최적화, 상용개발에 더 적합
- 현재 모델을 graph형태로 변경하기 위해서는, eager를 disable하고 실행하면 됨
- 관련 예제 코드: [MNIST with Eager](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples/mnist)
- 위의 예제 코드는 checkpoints를 저장하고 불러올 수 있기 때문에, 상용에도 적합하다.

## 현재 활용하고 있는 코드의 변화
 
- 현재 사용하고 있는 데이터에서, 형태를 tf.data로 변경하는것을 추천 드립니다.
  - [참고링크 1](https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html)
  - [참고링크 2](https://www.tensorflow.org/programmers_guide/datasets)
- tf.layer.Conv2D()와 같은 Object-oriented 기능 활용을 추천 (Explict storage for variables
- 대부분의 모델이 eagar로 활용이 가능하지만, dynamic 모델에 대한 control flow같은 경우는 추가로 검토가 필요
- tfe.enable_eagar_execution()을 활용하면, 끌수가 없기 때문에 Python 세션을 재시작 추천

## 향후 To-Do

- 11/29 블로그를 지속적으로 백업하면서, 추가 개선 방향을 고민 할 것
 1. 데이터 전처리
 2. 제 3의 모델 (Conv Seq2Seq) 테스트
 3. 기존의 심심이 데이터에서 Generation 한 것이 의미가 있어 보임 -> 블로그도 Generation 후, 실제 의마기 있는 문장에 대한 데이터 추출로 보강